In [13]:
import numpy as np
import pandas as pd

mean_by_years = pd.read_csv('../Predictions/submission_mean_by_years.csv')
rolling_avg = pd.read_csv('../Predictions/submission_rolling_avg.csv')
xgboost = pd.read_csv('../Predictions/submission_xgboost.csv')
random_forest = pd.read_csv('../Predictions/submission_random_forest.csv')
prophet = pd.read_csv('../Predictions/submission_prophet.csv')
sarimax = pd.read_csv('../Predictions/submission_sarimax.csv')

# Sort just in case
mean_by_years.sort_values(by='id')
rolling_avg.sort_values(by='id')
xgboost.sort_values(by='id')
random_forest.sort_values(by='id')
prophet.sort_values(by='id')
sarimax.sort_values(by='id')

# Rename sales columns for each model
mean_by_years = mean_by_years.rename({'sales':'mean_by_years'},axis=1)
rolling_avg = rolling_avg.rename({'sales':'rolling_avg'},axis=1)
xgboost = xgboost.rename({'sales':'xgboost'},axis=1)
random_forest = random_forest.rename({'sales':'random_forest'},axis=1)
prophet = prophet.rename({'sales':'prophet'},axis=1)
sarimax = sarimax.rename({'sales':'sarimax'},axis=1)

In [16]:
all_submissions = pd.merge(mean_by_years, rolling_avg, on='id')
all_submissions = pd.merge(all_submissions,xgboost,on='id')
all_submissions = pd.merge(all_submissions,random_forest,on='id')
all_submissions = pd.merge(all_submissions,prophet,on='id')
all_submissions = pd.merge(all_submissions,sarimax,on='id')
all_submissions

,id,mean_by_years,rolling_avg,xgboost,random_forest,prophet,sarimax
0,3000888,3.1000,4.965517,5.150004,2.780000,4.643406,4.384954
1,3000889,0.0000,0.000000,0.000003,0.000000,0.000000,0.000000
2,3000890,2.4000,3.310345,5.657823,6.200000,6.417149,6.343135
3,3000891,1915.7000,2060.068966,2565.031903,2536.290000,2310.096747,2264.613971
4,3000892,0.0000,0.034483,0.082563,0.080000,0.452525,0.067759
...,...,...,...,...,...,...,...
28507,3029395,390.4212,437.880281,320.287099,321.461038,337.873128,226.134574
28508,3029396,90.4535,120.867428,138.182761,100.778790,115.803888,105.270215
28509,3029397,882.6474,1606.105500,663.876308,1221.725300,1384.561281,1311.292960
28510,3029398,16.4000,150.071429,41.371271,86.140000,132.045618,140.092297


In [23]:
weight_mean_by_years = 0
weight_rolling = 1/3
weight_xgboost = 0
weight_random_forest = 0
weight_prophet = 1/3
weight_sarimax = 1/3

In [24]:
all_submissions['sales'] = (
    weight_mean_by_years*all_submissions['mean_by_years'] + 
    weight_rolling*all_submissions['rolling_avg'] +
    weight_xgboost*all_submissions['xgboost'] +
    weight_random_forest*all_submissions['random_forest'] +
    weight_prophet*all_submissions['prophet'] +
    weight_sarimax*all_submissions['sarimax']
)
all_submissions

,id,mean_by_years,rolling_avg,xgboost,random_forest,prophet,sarimax,sales
0,3000888,3.1000,4.965517,5.150004,2.780000,4.643406,4.384954,4.664626
1,3000889,0.0000,0.000000,0.000003,0.000000,0.000000,0.000000,0.000000
2,3000890,2.4000,3.310345,5.657823,6.200000,6.417149,6.343135,5.356876
3,3000891,1915.7000,2060.068966,2565.031903,2536.290000,2310.096747,2264.613971,2211.593228
4,3000892,0.0000,0.034483,0.082563,0.080000,0.452525,0.067759,0.184922
...,...,...,...,...,...,...,...,...
28507,3029395,390.4212,437.880281,320.287099,321.461038,337.873128,226.134574,333.962661
28508,3029396,90.4535,120.867428,138.182761,100.778790,115.803888,105.270215,113.980510
28509,3029397,882.6474,1606.105500,663.876308,1221.725300,1384.561281,1311.292960,1433.986580
28510,3029398,16.4000,150.071429,41.371271,86.140000,132.045618,140.092297,140.736448


In [ ]:
submission = all_submissions[['id','sales']]

,id,sales
0,3000888,4.664626
1,3000889,0.000000
2,3000890,5.356876
3,3000891,2211.593228
4,3000892,0.184922
...,...,...
28507,3029395,333.962661
28508,3029396,113.980510
28509,3029397,1433.986580
28510,3029398,140.736448


In [ ]:
# Run to create submission file
# Naming convention is submission_weighted_ followed by relative weights given to each model
# models are in order of their original scores, so:
# mean_by_year, rolling_avg, xgboost, random_forest, prophet, sarimax
submission.to_csv("../Predictions/submission_weighted_0-1-0-0-1-1.csv", index = False)